В процессе выполнения этого задания использовалось несколько ноутбуков для одновременной работы с несколькими моделями. По окончанию работы все было собрано в один файл по этой причине некоторые ячейки отмечены как не запущеные.

### Данные

Для обучения использовались отзывы пользователей о мобильных телефонах взятых с одного достаточно известного сайта  
С учетом того что в сыром виде тренировочные дванные имеют 5 классов была произведена предобратотка 
 - 1 и 2 были маркированы как негативные
 - 4 и 5 были маркированы как позитавные
 - 3 был отброшен
 
И того получилося примерно по 2к записей в каждом из 2 финальных классов.

### Результаты

 * LinearSVC       - 0.98 и 30 место в рейтинговой таблице
 * RidgeClassifier - 1.00 и 11 место в рейтисноговой таблице

In [1]:
%pylab inline

import warnings

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from nltk import WordNetLemmatizer, word_tokenize
from sklearn.linear_model import RidgeClassifier
from nltk.stem.snowball import RussianStemmer
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

warnings.simplefilter("ignore")

Populating the interactive namespace from numpy and matplotlib


In [2]:
def convert_y_to_number(y):
    return [0 if c == 'neg' else 1 for c in y]

In [3]:
def convert_predictions_to_str(predictions):
    return ['neg' if p == 0 else 'pos' for p in predictions]

In [ ]:
rs = RussianStemmer()

def stem_data(data):
    tokens_list = [word_tokenize(raw) for raw in data]
    tokens_list = [[rs.stem(t) for t in tokens] for tokens in tokens_list]
    
    return [' '.join(x) for x in tokens_list]

In [53]:
def submission_fit(model, X, y):
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X, y, cv=kf)
    
    print(score)
    print(score.mean())
    print(score.std())
    
    model.fit(X, y)
    
    competition_data = pd.read_csv('data/temp.csv', delimiter='\t', header=None)
    competition_data.columns = ['text']
    predictions = model.predict(stem_data(competition_data.text))
    
    df = pd.DataFrame({'y': convert_predictions_to_str(predictions)})
    df.index.name = 'Id'
    df.to_csv('data/results.csv')

In [5]:
def get_train_data():
    train_data = pd.read_csv('data/train_data.csv', usecols=['class', 'text'])

    return train_data.text.values, convert_y_to_number(train_data['class'].values)

In [6]:
models = [LogisticRegression, SGDClassifier, Perceptron, PassiveAggressiveClassifier, RidgeClassifier, LinearSVC, SVC, MultinomialNB, KNeighborsClassifier, NearestCentroid, RandomForestClassifier]

In [7]:
X, y = get_train_data()
stop_words = ['Достоинства', 'Недостатки', 'Комментарий']
X_stemed = stem_data(X)

In [55]:
pipe = Pipeline([
    ('vectorizer', CountVectorizer(stop_words=stop_words, analyzer='word', ngram_range=(1, 3), max_features=20000, max_df=0.5)), 
    ('tfidf', TfidfTransformer(norm='l2', use_idf=True)),
    ('classifier', RidgeClassifier(alpha=1, random_state=42))
])

param_grid = [
    {
        'vectorizer__min_df': [0.1, 0.2, 0.3]
        'vectorizer__max_df': [0.5, 0.6, 0.7]
        'vectorizer__max_features': [10000, 15000, 20000, 25000],
        'vectorizer__ngram_range': [(1, 2), (1, 3), (1, 4), (1, 5)],
        'vectorizer__analyzer': ['word', 'char', 'char_wb']
        'tfidf__norm': ['l1', 'l2'],
        'tfidf__use_idf': [True, False]
        'classifier__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
    }
]

kf = KFold(n_splits=5, shuffle=True)

grid = GridSearchCV(pipe, cv=kf, n_jobs=1, param_grid=param_grid, scoring='accuracy')
grid.fit(X_stemed, y)

print(grid.best_score_)
print()
for k in grid.best_params_:
    print(k, " : ", grid.best_params_[k])

In [54]:
submission_fit(pipe, X_stemed, y)

[ 0.88636364  0.87987013  0.88732394  0.88190683  0.8992416 ]
0.886941227786
0.00673980647505
